In [7]:
# Import Pandas
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np


In [8]:
df = pd.read_excel('DFI -Ticketing.xlsx', sheet_name=None)

df = pd.DataFrame(df['Export - Tickets'])


In [10]:
df = df.dropna(subset=['Incident-Subject','Ticket-Description'])


In [11]:
df['Incident-Subject'][0]

" Bonsoir  \xa0  Notre outil de supervision a remonté l'alerte suivante:  \xa0  Client:  Serveur: FRDAPLANIF1  Adresse IP: 170.237.25.77  Service: Service: _SMAOpConMinosService_\xa0  Note: Serveur Windows  \xa0  Retour du check: SMAOpConMinosService: Stopped  \xa0  \xa0  Cordialement, Adil  "

In [12]:
def add_questions(df,question):

  tickets_questions = df['Incident-Subject'].copy()
  tickets_questions = tickets_questions.to_frame()

  x=str(question)
  
  x = pd.DataFrame([[x]], columns=['Incident-Subject'])
  tickets_questions=tickets_questions.append(x, ignore_index=True)

  return tickets_questions



In [13]:
def answer_ticket(df):

  #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
  tfidf = TfidfVectorizer()

  #Construct the required TF-IDF matrix by fitting and transforming the data
  tfidf_matrix = tfidf.fit_transform(df)
  cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
  print(cosine_similarity.shape)
  sim_scores = list(enumerate(cosine_similarity[-1]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  #print(len(sim_scores))
  max_scores_loc = [i[0] for i in sim_scores if i[0] < len(sim_scores)-1]
  max_scores_loc = max_scores_loc[0:5]

  #print(max_scores_loc)

  return max_scores_loc


In [14]:
question = df['Incident-Subject'][1]
question =  'Bonsoir     Notre outil de supervision a remonté l''alerte suivante:     Client:  Serveur: FRDAPLANIF1  Adresse IP: 170.237.25.77  Service: Service: _SMAOpConMinosService_   Note: Serveur Windows     Retour du check: SMAOpConMinosService: Stopped        Cordialement, Adil'

tickets_questions = add_questions(df,question)

max_scores_loc = answer_ticket(tickets_questions['Incident-Subject'])

df['Ticket-Description'][max_scores_loc]

(10001, 10001)


0        Bonsoir,     Le service Minos a redémarré aut...
934      Bonjour,     le service est démarré, je clôtu...
978      Bonsoir,        Service: "SMAOpConMinosServic...
1906     Suite a derniere note : Service OK  Davy Mons...
4551     Bonjour,     La sauvegarde du serveur FRNS001...
Name: Ticket-Description, dtype: object